<a href="https://colab.research.google.com/github/gkondayya/ACP2021/blob/main/PINN_(TF2_0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Dependencies

## Getting the datasets

In [ ]:
!git clone https://github.com/maziarraissi/PINNs

fatal: destination path 'PINNs' already exists and is not an empty directory.


## Setting up modules

TeX packages

In [ ]:
# !sudo apt-get -qq install texlive-fonts-recommended texlive-fonts-extra dvipng

Pip modules

In [ ]:
!pip install tensorflow-gpu pyDOE

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 6.5 kB/s 
     |████████████████████████████████| 438 kB 12.7 MB/s 
     |████████████████████████████████| 5.8 MB 58.6 MB/s 
     |████████████████████████████████| 1.6 MB 9.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5

## Imports, config, and utils

In [ ]:

class NeuralNetwork(object):
    def __init__(self, hp, logger, ub, lb):

        layers = hp["layers"]

        # Setting up the optimizers with the hyper-parameters
        self.nt_config = Struct()
        self.nt_config.learningRate = hp["nt_lr"]
        self.nt_config.maxIter = hp["nt_epochs"]
        self.nt_config.nCorrection = hp["nt_ncorr"]
        self.nt_config.tolFun = 1.0 * np.finfo(float).eps
        self.tf_epochs = hp["tf_epochs"]
        self.tf_optimizer = tf.keras.optimizers.Adam(
            learning_rate=hp["tf_lr"],
            beta_1=hp["tf_b1"],
            epsilon=hp["tf_eps"])

        self.dtype = "float64"
        # Descriptive Keras model
        tf.keras.backend.set_floatx(self.dtype)
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.InputLayer(input_shape=(layers[0],)))
        self.model.add(tf.keras.layers.Lambda(
            lambda X: 2.0*(X - lb)/(ub - lb) - 1.0))
        for width in layers[1:-1]:
            self.model.add(tf.keras.layers.Dense(
                width, activation=tf.nn.tanh,
                kernel_initializer="glorot_normal"))
        self.model.add(tf.keras.layers.Dense(
                layers[-1], activation=None,
                kernel_initializer="glorot_normal"))

        # Computing the sizes of weights/biases for future decomposition
        self.sizes_w = []
        self.sizes_b = []
        for i, width in enumerate(layers):
            if i != 1:
                self.sizes_w.append(int(width * layers[1]))
                self.sizes_b.append(int(width if i != 0 else layers[1]))

        self.logger = logger

    # Defining custom loss
    # @tf.function
    def loss(self, u, u_pred):
        return tf.reduce_mean(tf.square(u - u_pred))

    # @tf.function
    def grad(self, X, u):
        with tf.GradientTape() as tape:
            loss_value = self.loss(u, abs(self.model(X)))
        grads = tape.gradient(loss_value, self.wrap_training_variables())
        return loss_value, grads

    def wrap_training_variables(self):
        var = self.model.trainable_variables
        return var

    def get_params(self, numpy=False):
        return []

    def get_weights(self, convert_to_tensor=True):
        w = []
        for layer in self.model.layers[1:]:
            weights_biases = layer.get_weights()
            weights = weights_biases[0].flatten()
            biases = weights_biases[1]
            w.extend(weights)
            w.extend(biases)
        if convert_to_tensor:
            w = self.tensor(w)
        return w

    def set_weights(self, w):
        for i, layer in enumerate(self.model.layers[1:]):
            start_weights = sum(self.sizes_w[:i]) + sum(self.sizes_b[:i])
            end_weights = sum(self.sizes_w[:i+1]) + sum(self.sizes_b[:i])
            weights = w[start_weights:end_weights]
            w_div = int(self.sizes_w[i] / self.sizes_b[i])
            weights = tf.reshape(weights, [w_div, self.sizes_b[i]])
            biases = w[end_weights:end_weights + self.sizes_b[i]]
            weights_biases = [weights, biases]
            layer.set_weights(weights_biases)

    def get_loss_and_flat_grad(self, X, u):
        def loss_and_flat_grad(w):
            with tf.GradientTape() as tape:
                self.set_weights(w)
                loss_value = self.loss(u, self.model(X))
            grad = tape.gradient(loss_value, self.wrap_training_variables())
            grad_flat = []
            for g in grad:
                grad_flat.append(tf.reshape(g, [-1]))
            grad_flat = tf.concat(grad_flat, 0)
            return loss_value, grad_flat

        return loss_and_flat_grad

    def tf_optimization(self, X_u, u):
        self.logger.log_train_opt("Adam")
        for epoch in range(self.tf_epochs):
            loss_value = self.tf_optimization_step(X_u, u)
            self.logger.log_train_epoch(epoch, loss_value)

    # @tf.function
    def tf_optimization_step(self, X_u, u):
        loss_value, grads = self.grad(X_u, u)
        self.tf_optimizer.apply_gradients(
                zip(grads, self.wrap_training_variables()))
        return loss_value

    def nt_optimization(self, X_u, u):
        self.logger.log_train_opt("LBFGS")
        loss_and_flat_grad = self.get_loss_and_flat_grad(X_u, u)
        # tfp.optimizer.lbfgs_minimize(
        #   loss_and_flat_grad,
        #   initial_position=self.get_weights(),
        #   num_correction_pairs=nt_config.nCorrection,
        #   max_iterations=nt_config.maxIter,
        #   f_relative_tolerance=nt_config.tolFun,
        #   tolerance=nt_config.tolFun,
        #   parallel_iterations=6)
        self.nt_optimization_steps(loss_and_flat_grad)

    def nt_optimization_steps(self, loss_and_flat_grad):
        lbfgs(loss_and_flat_grad,
              self.get_weights(),
              self.nt_config, Struct(), True,
              lambda epoch, loss, is_iter:
              self.logger.log_train_epoch(epoch, loss, "", is_iter))

    def fit(self, X_u, u):
        self.logger.log_train_start(self)

        # Creating the tensors
        X_u = self.tensor(X_u)
        u = self.tensor(u)

        # Optimizing
        self.tf_optimization(X_u, u)
        self.nt_optimization(X_u, u)

        self.logger.log_train_end(self.tf_epochs + self.nt_config.maxIter)

    def predict(self, X_star):
        u_pred = self.model(X_star)
        return u_pred.numpy()

    def summary(self):
        return self.model.summary()

    def tensor(self, X):
        return tf.convert_to_tensor(X, dtype=self.dtype)

In [ ]:
!pip install pyDOE
import sys
import os
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp

# Manually making sure the numpy random seeds are "the same" on all devices
np.random.seed(1234)
tf.random.set_seed(1234)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


custom_lbfgs.py

In [ ]:
# Adapted from https://github.com/yaroslavvb/stuff/blob/master/eager_lbfgs/eager_lbfgs.py

import tensorflow as tf
import numpy as np
import time

# Time tracking functions
global_time_list = []
global_last_time = 0
def reset_time():
  global global_time_list, global_last_time
  global_time_list = []
  global_last_time = time.perf_counter()
  
def record_time():
  global global_last_time, global_time_list
  new_time = time.perf_counter()
  global_time_list.append(new_time - global_last_time)
  global_last_time = time.perf_counter()
  #print("step: %.2f"%(global_time_list[-1]*1000))

def last_time():
  """Returns last interval records in millis."""
  global global_last_time, global_time_list
  if global_time_list:
    return 1000 * global_time_list[-1]
  else:
    return 0

def dot(a, b):
  """Dot product function since TensorFlow doesn't have one."""
  return tf.reduce_sum(a*b)

def verbose_func(s):
  print(s)

final_loss = None
times = []
def lbfgs(opfunc, x, config, state, do_verbose, log_fn):
  """port of lbfgs.lua, using TensorFlow eager mode.
  """

  if config.maxIter == 0:
    return

  global final_loss, times
  
  maxIter = config.maxIter
  maxEval = config.maxEval or maxIter*1.25
  tolFun = config.tolFun or 1e-5
  tolX = config.tolX or 1e-19
  nCorrection = config.nCorrection or 100
  lineSearch = config.lineSearch
  lineSearchOpts = config.lineSearchOptions
  learningRate = config.learningRate or 1
  isverbose = config.verbose or False

  # verbose function
  if isverbose:
    verbose = verbose_func
  else:
    verbose = lambda x: None

    # evaluate initial f(x) and df/dx
  f, g = opfunc(x)

  f_hist = [f]
  currentFuncEval = 1
  state.funcEval = state.funcEval + 1
  p = g.shape[0]

  # check optimality of initial point
  tmp1 = tf.abs(g)
  if tf.reduce_sum(tmp1) <= tolFun:
    verbose("optimality condition below tolFun")
    return x, f_hist

  # optimize for a max of maxIter iterations
  nIter = 0
  times = []
  while nIter < maxIter:
    start_time = time.time()
    
    # keep track of nb of iterations
    nIter = nIter + 1
    state.nIter = state.nIter + 1

    ############################################################
    ## compute gradient descent direction
    ############################################################
    if state.nIter == 1:
      d = -g
      old_dirs = []
      old_stps = []
      Hdiag = 1
    else:
      # do lbfgs update (update memory)
      y = g - g_old
      s = d*t
      ys = dot(y, s)
      
      if ys > 1e-10:
        # updating memory
        if len(old_dirs) == nCorrection:
          # shift history by one (limited-memory)
          del old_dirs[0]
          del old_stps[0]

        # store new direction/step
        old_dirs.append(s)
        old_stps.append(y)

        # update scale of initial Hessian approximation
        Hdiag = ys/dot(y, y)

      # compute the approximate (L-BFGS) inverse Hessian 
      # multiplied by the gradient
      k = len(old_dirs)

      # need to be accessed element-by-element, so don't re-type tensor:
      ro = [0]*nCorrection
      for i in range(k):
        ro[i] = 1/dot(old_stps[i], old_dirs[i])
        

      # iteration in L-BFGS loop collapsed to use just one buffer
      # need to be accessed element-by-element, so don't re-type tensor:
      al = [0]*nCorrection

      q = -g
      for i in range(k-1, -1, -1):
        al[i] = dot(old_dirs[i], q) * ro[i]
        q = q - al[i]*old_stps[i]

      # multiply by initial Hessian
      r = q*Hdiag
      for i in range(k):
        be_i = dot(old_stps[i], r) * ro[i]
        r += (al[i]-be_i)*old_dirs[i]
        
      d = r
      # final direction is in r/d (same object)

    g_old = g
    f_old = f
    
    ############################################################
    ## compute step length
    ############################################################
    # directional derivative
    gtd = dot(g, d)

    # check that progress can be made along that direction
    if gtd > -tolX:
      verbose("Can not make progress along direction.")
      break

    # reset initial guess for step size
    if state.nIter == 1:
      tmp1 = tf.abs(g)
      t = min(1, 1/tf.reduce_sum(tmp1))
    else:
      t = learningRate


    # optional line search: user function
    lsFuncEval = 0
    if lineSearch and isinstance(lineSearch) == types.FunctionType:
      # perform line search, using user function
      f,g,x,t,lsFuncEval = lineSearch(opfunc,x,t,d,f,g,gtd,lineSearchOpts)
      f_hist.append(f)
    else:
      # no line search, simply move with fixed-step
      x += t*d
      
      if nIter != maxIter:
        # re-evaluate function only if not in last iteration
        # the reason we do this: in a stochastic setting,
        # no use to re-evaluate that function here
        f, g = opfunc(x)
        lsFuncEval = 1
        f_hist.append(f)


    # update func eval
    currentFuncEval = currentFuncEval + lsFuncEval
    state.funcEval = state.funcEval + lsFuncEval

    ############################################################
    ## check conditions
    ############################################################
    if nIter == maxIter:
      break

    if currentFuncEval >= maxEval:
      # max nb of function evals
      verbose('max nb of function evals')
      break

    tmp1 = tf.abs(g)
    if tf.reduce_sum(tmp1) <=tolFun:
      # check optimality
      verbose('optimality condition below tolFun')
      break
    
    tmp1 = tf.abs(d*t)
    if tf.reduce_sum(tmp1) <= tolX:
      # step size below tolX
      verbose('step size below tolX')
      break

    if tf.abs(f-f_old) < tolX:
      # function value changing less than tolX
      verbose('function value changing less than tolX'+str(tf.abs(f-f_old)))
      break

    if do_verbose:
      log_fn(nIter, f.numpy(), True)
      #print("Step %3d loss %6.5f msec %6.3f"%(nIter, f.numpy(), last_time()))
      record_time()
      times.append(last_time())

    if nIter == maxIter - 1:
      final_loss = f.numpy()


  # save state
  state.old_dirs = old_dirs
  state.old_stps = old_stps
  state.Hdiag = Hdiag
  state.g_old = g_old
  state.f_old = f_old
  state.t = t
  state.d = d

  return x, f_hist, currentFuncEval

# dummy/Struct gives Lua-like struct object with 0 defaults
class dummy(object):
  pass

class Struct(dummy):
  def __getattribute__(self, key):
    if key == '__dict__':
      return super(dummy, self).__getattribute__('__dict__')
    return self.__dict__.get(key, 0)

In [ ]:

class Logger(object):
  def __init__(self, frequency=10):
    print("TensorFlow version: {}".format(tf.__version__))
    print("Eager execution: {}".format(tf.executing_eagerly()))
    print("GPU-accerelated: {}".format(tf.test.is_gpu_available()))

    self.start_time = time.time()
    self.frequency = frequency

  def __get_elapsed(self):
    return datetime.fromtimestamp(time.time() - self.start_time).strftime("%M:%S")

  def __get_error_u(self):
    return self.error_fn()

  def set_error_fn(self, error_fn):
    self.error_fn = error_fn
  
  def log_train_start(self, model):
    print("\nTraining started")
    print("================")
    self.model = model
    print(self.model.summary())

  def log_train_epoch(self, epoch, loss, custom="", is_iter=False):
    # if epoch % self.frequency == 0:
      print(f"{'nt_epoch' if is_iter else 'tf_epoch'} = {epoch:6d}  elapsed = {self.__get_elapsed()}  loss = {loss:.4e}  error = {self.__get_error_u():.4e}  " + custom)

  def log_train_opt(self, name):
    # print(f"tf_epoch =      0  elapsed = 00:00  loss = 2.7391e-01  error = 9.0843e-01")
    print(f"—— Starting {name} optimization ——")

  def log_train_end(self, epoch, custom=""):
    print("==================")
    print(f"Training finished (epoch {epoch}): duration = {self.__get_elapsed()}  error = {self.__get_error_u():.4e}  " + custom)

def prep_data(N_b=None, N_f=None, ub=None, lb=None, noise=0.0, N_0=None):
    x = np.linspace(lb[0],ub[0],501)
    x = np.reshape(x,(501,1)) #making column vector

    t = np.linspace(lb[1],ub[1],501)
    t = np.reshape(t,(501,1)) #making column vector

    Exact_u = np.loadtxt('InterpolatedFlux-G1.csv',delimiter=",")
    Exact_v = np.loadtxt('InterpolatedFlux-G2.csv',delimiter=",")

    # for i in range(1001):
    #     for j in range(1001):
    #         Exact_u[i,j] = F * np.cos((np.pi*x[i])/a)  * np.cos((np.pi*t[j])/a)
    #         Exact_v[i,j] = (F * np.cos((np.pi*x[i])/a)*np.cos((np.pi*t[j])/a))*s1/(r2+d2*((np.pi/a)**2)) #* (F * np.cos((np.pi*t[j])/a))*s1/(r2+d2*((np.pi/a)**2))

    X, T = np.meshgrid(x,t)
    
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact_u.T.flatten()[:,None]
    v_star = Exact_v.T.flatten()[:,None]
    
    ###########################
    
    idx_x = np.random.choice(x.shape[0], N_0, replace=False)
    idx_t = np.random.choice(t.shape[0], N_0, replace=False)
    x0 = x[idx_x,:]
    t0 = t[idx_t,:]
    u0 = np.zeros(N_0)
    v0 = np.zeros(N_0)
    for i in range(N_0):
            u0[i] = Exact_u[idx_x[i],idx_t[i]]
            v0[i] = Exact_v[idx_x[i],idx_t[i]]
    u0 = np.reshape(u0,(N_0,1))
    v0 = np.reshape(v0,(N_0,1))
    print(u0[5],Exact_u[idx_x[5],idx_t[5]],x[idx_x[5],:],t[idx_t[5],:],idx_x[5],idx_t[5])
    
    ############################

    x1_b = np.random.choice(x.shape[0],N_b,replace = False)
    x2_b = np.random.choice(t.shape[0],N_b,replace = False)
    t1_b = np.random.choice(t.shape[0],N_b,replace = False)
    t2_b = np.random.choice(t.shape[0],N_b,replace = False)
    xb0 = np.zeros(N_b,dtype = int)
    xb1 = np.full(N_b,500,dtype = int)
    x_b0 = np.hstack((xb0,xb1))
    x_b1 = np.hstack((x_b0,x1_b))

    t_b0 = np.hstack((xb0,xb1))
    t_b1 = np.hstack((t_b0,t1_b))

    ixb = np.hstack((x_b1,x2_b))
    itb_ = np.hstack((t_b1,t2_b))
    itb = np.flip(itb_)

#     #### new BC
#     ixb = [0,0,500,500]
#     itb = [0,500,0,500]
    xb = x[ixb,:]
    tb = t[itb,:]
    u_b = np.reshape(Exact_u[ixb,itb],(N_b*4,1))
    v_b = np.reshape(Exact_v[ixb,itb],(N_b*4,1))
    # u_b = np.zeros((N_b*4,1))
    # v_b = np.zeros((N_b*4,1))
    

  

    X_f = lb + (ub-lb)*lhs(2, N_f)
    return x, t, X, T, Exact_u, Exact_v, \
    X_star, u_star, v_star, X_f, \
    ub, lb, u_b, v_b, xb, tb, x0, t0, u0, v0
            

# 1. Continuous Inference

In [ ]:
#CONSTANTS

a = 2*60.5
div = 5000

import sys
import json
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
import time
from pyDOE import lhs
import json
import tensorflow as tf
import time
from datetime import datetime

# Manually making sure the numpy random seeds are "the same" on all devices
np.random.seed(1234)
tf.random.set_seed(1234)


# HYPER PARAMETERS

hp = {}

# Doman bounds
lb = np.array([(-1)*a/2, (-1)*a/2])
ub = np.array([a/2, a/2])
# Data size on the initial condition solution
hp["N_0"] = 100
# Collocation points on the boundaries
hp["N_b"] = 100
# Collocation points on the domain
hp["N_f"] = 15000
# DeepNN topology (2-sized input [x t], 4 hidden layer of 100-width, 2-sized output [u, v])
hp["layers"] = [2, 100, 100, 100, 100, 2]
# Setting up the TF SGD-based optimizer (set tf_epochs=0 to cancel it)
hp["tf_epochs"] = 2000
hp["tf_lr"] = 0.5
hp["tf_b1"] = 0.99
hp["tf_eps"] = 1e-1
# Setting up the quasi-newton LBGFS optimizer (set nt_epochs=0 to cancel it)
hp["nt_epochs"] = 250
hp["nt_lr"] = 1.2
hp["nt_ncorr"] = 50
hp["log_frequency"] = 10


# %% DEFINING THE MODEL


class SchrodingerInformedNN(NeuralNetwork):
    def __init__(self, hp, logger, X_f, x0, t0, u0, v0, xb, tb, ub, lb, u_b, v_b):
        super().__init__(hp, logger, ub, lb)
        X0 = np.concatenate((x0, t0), 1)
        X_b = np.concatenate((xb, tb), 1)
        
        self.u0 = u0
        self.v0 = v0
        self.u_b = u_b
        self.v_b = v_b

        self.X_b = self.tensor(X_b)

        # Separating the collocation coordinates
        self.x_f = self.tensor(X_f[:, 0:1])
        self.t_f = self.tensor(X_f[:, 1:2])

    def splitter(self,inpt):
        batch = int(hp["N_f"])/div
        u_batch = []
        for i in range(int(batch)):
            u_batch.append(inpt[i*5000:(i+1)*5000])
        u_batch = np.array(u_batch)
        return u_batch

    # Decomposes the multi-output into the complex values and spatial derivatives
    def uvx_model(self, X):
        x = X[:, 0:1]
        t = X[:, 1:2]
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            tape.watch(t)
            Xtemp = tf.concat([x, t], axis=1)

            h = self.model(Xtemp)
            
            
            #ensuring flux is non negative always
            h = abs(h)
            
            u = h[:, 0:1]
            v = h[:, 1:2]

        u_x = tape.gradient(u, x)
        v_x = tape.gradient(v, x)
        u_t = tape.gradient(u, t)
        v_t = tape.gradient(v, t)
        del tape

        return u, v, u_x, v_x, u_t, v_t
    def _sum(arr): 
        sum=0
        for i in arr:
            sum = sum + i
        return(sum) 

    # The actual PINN
    def f_model(self):
        global k_temp
        global u_sum, v_sum
        u_sum_old = u_sum
        v_sum_old = v_sum
        # Using the new GradientTape paradigm of TF2.0,
        # which keeps track of operations to get the gradient at runtime
        with tf.GradientTape(persistent=True) as tape:
            # Watching the two inputs we’ll need later, x and t
            tape.watch(self.x_f)
            tape.watch(self.t_f)
            # Packing together the inputs
            X_f = tf.concat([self.x_f, self.t_f], axis=1)

            # Getting the prediction
            u, v, u_x, v_x, u_t, v_t = self.uvx_model(X_f)

        u_sum = sum(u)
        v_sum = sum(v)
        print(v_sum)
        
        k_temp = 0.911060 
        # k_temp = k_temp +  u_sum*v_sum/u_sum_old*v_sum_old

        # Getting the other derivatives
        u_xx = tape.gradient(u_x, self.x_f)
        v_xx = tape.gradient(v_x, self.x_f)
        u_tt = tape.gradient(u_t, self.t_f)
        v_tt = tape.gradient(v_t, self.t_f)

        # Letting the tape go
        del tape

        #split into batches of 5000 div
        
        u_batch = self.splitter(u)
        v_batch = self.splitter(v)
        u_xx_batch = self.splitter(u_xx)
        v_xx_batch = self.splitter(v_xx)
        u_tt_batch = self.splitter(u_tt)
        v_tt_batch = self.splitter(v_tt)
        d1_batch = self.splitter(d1_)
        d2_batch = self.splitter(d2_)
        r1_batch = self.splitter(r1_)
        r2_batch = self.splitter(r2_)
        s1_batch = self.splitter(s1_)
        f1_batch = self.splitter(f1_)
        f2_batch = self.splitter(f2_)
        F_batch =  self.splitter(F_)
        nu_batch =  self.splitter(nu_)
        k_batch =  self.splitter(k_)



        f_u = []
        f_v = []

        batch = int(hp["N_f"])/div
        for i in range(int(batch)):
          
            delta_u = u_xx_batch[i,:,:] + u_tt_batch[i,:,:]
            delta_v = v_xx_batch[i,:,:] + v_tt_batch[i,:,:]
            f_u.extend( (d1_batch[i,:] * delta_u)/2 - r1_batch[i,:]*u_batch[i,:,:] + (1/k_temp)*(nu_batch[i,:]*f1_batch[i,:]*u_batch[i,:,:] + nu_batch[i,:]*f2_batch[i,:]*v_batch[i,:,:]) )
            f_v.extend( (d2_batch[i,:] * delta_v)/2 - r2_batch[i,:]*v_batch[i,:,:] + s1_batch[i,:]*u_batch[i,:,:] ) #check
        return f_u, f_v, k_temp
    


    def loss(self, uv, uv_pred):
        u0_pred = uv_pred[:, 0:1]
        v0_pred = uv_pred[:, 1:2]
        u_b_pred, v_b_pred, _, _, _, _= \
                self.uvx_model(self.X_b)
        # f_u_pred, f_v_pred, k_temp = self.f_model()

        mse_0 = tf.reduce_mean(tf.square(self.u0 - u0_pred)) + tf.reduce_mean(tf.square(self.v0 - v0_pred)) 
        mse_b = tf.reduce_mean(tf.square(self.u_b - u_b_pred)) + \
            tf.reduce_mean(tf.square(self.v_b - v_b_pred))

        # mse_f = tf.reduce_mean(tf.square(f_u_pred)) + \
        #     tf.reduce_mean(tf.square(f_v_pred))


        tf.print(f"mse_0 {mse_0}    mse_b {mse_b}       k_temp   {k_temp}") #mse_f    {mse_f}
        return mse_0 + mse_b #+ mse_f

    def predict(self, X_star):
        h_pred = abs(self.model(X_star))
        u_pred = h_pred[:, 0:1]
        v_pred = h_pred[:, 1:2]
        return u_pred.numpy(), v_pred.numpy()

# %% TRAINING THE MODEL


# Getting the data
x, t, X, T, Exact_u, Exact_v, \
    X_star, u_star, v_star, X_f, \
    ub, lb, u_b, v_b, xb, tb, x0, t0, u0, v0  = prep_data(
        N_0 = hp["N_0"], N_b = hp["N_b"], N_f = hp["N_f"], ub = ub, lb = lb, noise=0.0)


def lister(value1, value2):
    temp = []
    for i in range(hp["N_f"]):
        
        if abs(X_f[i,0]) < (8.25) and abs(X_f[i,1]) < (8.25):
            temp.append(value1)
            print("hello")
        else:
            temp.append(value2)
            print("hella")
    temp = np.array(temp)    
    temp = np.reshape(temp,(hp["N_f"],1))
    return temp

# boolo = lister(0,1)
# d1_ = np.array([1.255, 1.2679])
# d2_ = np.array([0.211, 0.1902])
# r1_ = np.array([0.0336,  0.0349])
# r2_ = np.array([0.1003, 0.0704])
# s1_ = np.array([0.0253, 0.0277])
# f1_ = np.array([0.0046, 0.0046]) # 0.45
# f2_ = np.array([0.1091, 0.0868]) # 0.8
# F_ =  np.array([1,1])
# nu_ = np.array([1, 1])


d1_ = lister(1.255, 1.2679)
d2_ = lister( 0.211, 0.1902)
r1_ = lister(0.0336,  0.0349)
r2_ = lister(0.1003, 0.0704)
s1_ = lister(0.0253, 0.0277)
f1_ = lister(0.0046, 0.0046)
f2_ =  lister(0.1091, 0.0868)
nu_ =  lister(1,1)
F_ =  lister(1,1)


k_temp = 0 #0.911060
u_sum = 1
v_sum = 1

k_ = ( (r2_ + d2_*((np.pi/a)**2)) * nu_*f1_ + s1_*nu_*f2_) / ( (d1_*((np.pi/a)**2) + r1_) * (d2_ * ((np.pi/a)**2) + r2_))
print(np.shape(k_),np.shape(F_))


# Creating the model
logger = Logger(hp)

pinn = SchrodingerInformedNN(hp, logger, X_f, x0, t0, u0, v0, xb, tb, ub, lb, u_b, v_b)

# Defining the error function for the logger


def error():
    u_pred, v_pred = pinn.predict(X_star)
    error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
    error_v = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
    print('Error u: %e' % (error_u))
    print('Error v: %e' % (error_v))
    return error_u

logger.set_error_fn(error)

# Training the PINN
X0 = np.concatenate((x0, t0), 1) # (x0, 0)
X_b = np.concatenate((xb , tb), 1) # (ub[0], tb)
X_u_train = np.vstack([X0, X_b])
u_train = np.vstack([u0,u_b])
v_train = np.vstack([v0,v_b])
U_V_train = tf.concat([u_train, v_train], axis=1)
U_V0_train = tf.concat([u0, v0], axis=1)
print(np.shape(X0),np.shape(X_u_train),np.shape(u0),np.shape(v0),np.shape(v_train),np.shape(u_train),np.shape(U_V_train),np.shape(U_V0_train))
pinn.fit(X0,U_V0_train) #(X_u_train, U_V_train )

# Getting the model predictions, from the same (x,t) that the predictions were previously gotten from
u_pred, v_pred = pinn.predict(X_star)
error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
error_v = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
print('Error u: %e' % (error_u))
print('Error v: %e' % (error_v))

U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
V_pred = griddata(X_star, v_pred.flatten(), (X, T), method='cubic')


Streaming output truncated to the last 5000 lines.
mse_0 0.1109070154014722    mse_b 0.09568807229171772       k_temp   0
Error u: 2.553542e+01
Error v: 9.906687e+01
tf_epoch =   1001  elapsed = 03:45  loss = 2.0660e-01  error = 2.5535e+01  
mse_0 0.12922035001929455    mse_b 0.1315339903806747       k_temp   0
Error u: 2.515201e+01
Error v: 8.848198e+01
tf_epoch =   1002  elapsed = 03:45  loss = 2.6075e-01  error = 2.5152e+01  
mse_0 0.11016412746687272    mse_b 0.10696804799704986       k_temp   0
Error u: 2.337331e+01
Error v: 8.580565e+01
tf_epoch =   1003  elapsed = 03:45  loss = 2.1713e-01  error = 2.3373e+01  
mse_0 0.10538573123981838    mse_b 0.08618449749507495       k_temp   0
Error u: 2.359684e+01
Error v: 9.629368e+01
tf_epoch =   1004  elapsed = 03:45  loss = 1.9157e-01  error = 2.3597e+01  
mse_0 0.12488251619685538    mse_b 0.11881927916946926       k_temp   0
Error u: 2.424135e+01
Error v: 9.249292e+01
tf_epoch =   1005  elapsed = 03:46  loss = 2.4370e-01  error = 2.42